## API example

Что можно сделать с помощью __ml_api__?
1. Вернуть список доступных для обучения классов моделей
2. Обучить ML-модель с возможностью настройки гиперпараметров. При этом гиперпараметры для разных моделей могут быть разные. Количество классов моделей доступных для обучения = 4
3. Вернуть предсказание конкретной модели (система умеет хранить несколько обученных моделей)
4. Обучить заново и удалить уже обученные модели

С помощью __metrics__ можно посмотреть метрики, логируемые `prometheus_flask_exporter`

---

Сперва необходимо запустить утилиту `docker-compose`

In [1]:
import requests
import pandas as pd
from sklearn.datasets import load_iris, load_boston

Демонстрировать возможности API будем на датасетах _boston_ (задача регрессии) и _iris_ (задача классификации)

In [3]:
iris_data = load_iris()
iris_df = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris_df['species'] = iris_data.target
iris_json = iris_df.to_json()

boston_data = load_boston()
boston_df = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
boston_df['PRICE'] = boston_data.target
boston_json = boston_df.to_json()

Метод __get__ возвращает список доступных для обучения моделей, а точнее алгоритмов из `sklearn`. Это сделано для того, чтобы сразу определялся тип задачи – регрессия или классификация. Всего реализовано 4 класса моделей:
- Линейные модели
- Метод опорных векторов
- Дерево решений
- Случайный лес

In [5]:
resp = requests.get('http://0.0.0.0:8080/ml_api')
resp.json()

{'ml_models': ['LogisticRegression',
  'SVC',
  'DecisionTreeClassifier',
  'RandomForestClassifier',
  'Ridge',
  'SVR',
  'DecisionTreeRegressor',
  'RandomForestRegressor']}

Чтобы обучить модель, необходимо вызвать метод __post__ и передать в параметр __json__ всю необходимую информацию, а именно:
- name (str) – имя модели, под которым она будет сохраняться в `mlflow`, версия модели будет указана через /
- model (str) – модель, которую необходимо обучить (нужно выбрать из списка)
- data (json) – данные в формате json, на которых надо обучить модель (поддерживаются только количественные признаки, таргет – последний элемент файла)
- params (dict) – гиперпараметры модели, по умолчанию используются дефолтные значения соответствующего алгоритма в `sklearn`
- grid_search (bool) – надо ли делать подбор оптимальных значений гиперпараметров, по умолчанию нет
- param_grid (dict) – сетка для перебора значений гиперпараметров, по умолчанию используется заданная сетка

Внутри данного метода, помимо сохранения обучающих и тестовых данных (разбиение производится автоматически случайным образом в соотношении 7:3), обучения модели и сохранения модели, также рассчитываются метрики качества в зависимости от класса модели. В отличие от предыдущей версии, локально (в папку) сохраняются лишь данные, а модель, параметры и метрики логируются с помощью `mlflow`. 

Посмотреть результаты экспериментов можно по http://127.0.0.1:5050

Еще одно отличие от предыдущей версии приложения заключается в версионности моделей, то есть указывая одно и то же имя модели, можно создать несколько ее версий, например, с разными гиперпараметрами или на разных выборках. 

In [6]:
payload = {'model': 'RandomForestRegressor', 'data': boston_json, 'name': 'boston_rf'}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.text

(200, 'RandomForestRegressor is fitted and saved')

Чтобы посмотреть информацию обо всех обученных моделях, надо вызвать метод __get__ по адресу `/ml_api/all_models`. Для каждой модели хранится ее id (имя и версия модели), алгоритм (estimator), гиперпараметры, было ли обучение модели заново и была ли она удалена. Это своего рода файл истории использования API. Данные сохраняются в файлы, в названии которых указан id модели, в папке `./data`. 

In [8]:
response = requests.get('http://0.0.0.0:8080/ml_api/all_models')
response.json()

{'boston_rf_1': {'deleted': False,
  'model': 'RandomForestRegressor',
  'params': {'bootstrap': True,
   'ccp_alpha': 0.0,
   'criterion': 'mse',
   'max_depth': None,
   'max_features': 'auto',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   'min_impurity_split': None,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'n_estimators': 100,
   'n_jobs': None,
   'oob_score': False,
   'random_state': None,
   'verbose': 0,
   'warm_start': False},
  'retrained': False}}

Параметры модели, также как и сетку для их перебора, можно либо задать вручную, либо не задавать вовсе. В последнем случае используются дефолтные параметры. 

In [9]:
payload = {'model': 'RandomForestRegressor',
           'params': {'n_estimators': 200, 'max_depth': 10, 'min_samples_leaf': 3},
           'data': boston_json,
           'name': 'boston_rf'}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.text

(200, 'RandomForestRegressor is fitted and saved')

Можно убедиться, что создалась новая версия модели boston_rf

In [10]:
response = requests.get('http://0.0.0.0:8080/ml_api/all_models')
response.json()

{'boston_rf_1': {'deleted': False,
  'model': 'RandomForestRegressor',
  'params': {'bootstrap': True,
   'ccp_alpha': 0.0,
   'criterion': 'mse',
   'max_depth': None,
   'max_features': 'auto',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   'min_impurity_split': None,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'n_estimators': 100,
   'n_jobs': None,
   'oob_score': False,
   'random_state': None,
   'verbose': 0,
   'warm_start': False},
  'retrained': False},
 'boston_rf_2': {'deleted': False,
  'model': 'RandomForestRegressor',
  'params': {'bootstrap': True,
   'ccp_alpha': 0.0,
   'criterion': 'mse',
   'max_depth': 10,
   'max_features': 'auto',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   'min_impurity_split': None,
   'min_samples_leaf': 3,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'n_estimators': 200,
   'n_jobs': None,
   

API поддерживает работу только с количественными признаками. Если попытаться подать категориальные данные, выпадет ошибка 400. 

In [11]:
boston_df_cat = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
boston_df_cat['category'] = 'a'
boston_df_cat['PRICE'] = boston_data.target
boston_json_cat = boston_df_cat.to_json()

In [12]:
payload = {'model': 'RandomForestRegressor', 'data': boston_json_cat, 'name': 'boston_rf'}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.text

(400, 'Could not support categorical features')

Также ошибка выпадет, если пытаться строить модель не из списка доступных; использовать модель классификации для решения задачи регрессии и наоборот; передавать неправильный гиперпараметр (не поддерживаемый данным алгоритмом)

In [13]:
payload = {'model': 'RandomForestClassifier', 'data': boston_json, 'name': 'boston_rf'}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.text

(400, 'RandomForestClassifier can only be used for classification tasks')

In [14]:
payload = {'model': 'GradientBoostingRegressor', 'data': boston_json, 'name': 'boston_rf'}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.text

(404,
 "Can only train one of {'ml_models': ['LogisticRegression', 'SVC', 'DecisionTreeClassifier', 'RandomForestClassifier', 'Ridge', 'SVR', 'DecisionTreeRegressor', 'RandomForestRegressor']} models")

In [15]:
payload = {'model': 'RandomForestRegressor', 'params': {'learning_rate': 0.1}, 'data': boston_json, 
           'name': 'boston_rf'}
response = requests.post('http://0.0.0.0:8080/ml_api', json=payload)
response.status_code, response.text

(400,
 "RandomForestRegressor got an unexpected keyword argument 'learning_rate'")

Вернуть предсказания конкретной модели на трейне и тесте можно с помощью метода __get__ с указанием в пути имени и версии модели

In [16]:
response = requests.get('http://0.0.0.0:8080/ml_api/boston_rf/1')
response.json()

{'test_predictions': '[23.24, 30.94, 16.66, 23.95, 17.27, 22.38, 19.66, 14.54, 21.44, 21.06, 20.33, 19.29, 8.01, 21.97, 19.17, 24.86, 19.23, 8.41, 45.62, 15.74, 24.31, 23.85, 14.77, 23.76, 15.02, 15.22, 21.45, 14.04, 19.32, 21.1, 20.2, 23.34, 28.95, 20.56, 14.6, 16.32, 33.99, 19.06, 21.15, 23.95, 18.62, 30.0, 45.6, 19.48, 22.93, 13.65, 15.92, 24.31, 19.23, 27.93, 21.21, 33.66, 18.13, 26.11, 44.44, 21.79, 15.69, 31.97, 22.02, 20.88, 24.97, 33.81, 29.67, 18.82, 27.37, 16.98, 13.99, 22.99, 28.34, 15.93, 20.5, 28.87, 10.65, 21.63, 22.46, 7.01, 19.94, 46.05, 10.87, 12.41, 21.91, 12.14, 19.94, 9.38, 20.85, 27.37, 16.42, 23.33, 23.69, 17.83, 22.06, 7.45, 20.35, 19.29, 24.59, 19.9, 39.34, 12.02, 12.91, 12.32, 20.15, 24.1, 13.24, 20.33, 21.11, 12.15, 18.93, 24.71, 20.35, 23.22, 9.2, 16.06, 22.67, 25.3, 32.16, 14.82, 42.64, 16.3, 19.62, 24.12, 19.56, 23.92, 7.68, 20.78, 24.31, 21.93, 23.81, 34.9, 17.7, 44.8, 15.53, 23.59, 19.93, 18.59, 13.77, 20.99, 20.95, 31.79, 28.22, 17.32, 19.19, 24.6, 19.85

Под обучением заново понимается обучение модели на новых данных. Сделать это можно с помощью метода __put__. Необходимо в адресе запроса указать имя и версию модели, а в параметр __json__ передать новые данные (также в формате json). В результате создается новая версия моделии, а в словаре моделей ключ retrained примет значение версии модели, которая была переобучена.

In [17]:
response = requests.put('http://0.0.0.0:8080/ml_api/boston_rf/1', json=boston_df.sample(100).to_json())
response.status_code, response.text

(200, 'Model boston_rf of version 1 is re-fitted and saved')

In [20]:
response = requests.get('http://0.0.0.0:8080/ml_api/all_models')
list(response.json().items())[-1]

('boston_rf_3',
 {'deleted': False,
  'model': 'RandomForestRegressor',
  'params': {'bootstrap': True,
   'ccp_alpha': 0.0,
   'criterion': 'mse',
   'max_depth': None,
   'max_features': 'auto',
   'max_leaf_nodes': None,
   'max_samples': None,
   'min_impurity_decrease': 0.0,
   'min_impurity_split': None,
   'min_samples_leaf': 1,
   'min_samples_split': 2,
   'min_weight_fraction_leaf': 0.0,
   'n_estimators': 100,
   'n_jobs': None,
   'oob_score': False,
   'random_state': None,
   'verbose': 0,
   'warm_start': False},
  'retrained': '1'})

Чтобы удалить уже обученную модель, надо вызвать метод __delete__ и указать в адресе имя и версию модели. При корректном выполнении выпадает ошибка 204, а deleted в словаре меняет свое значение на True.

In [21]:
response = requests.delete('http://0.0.0.0:8080/ml_api/boston_rf/2')
response.status_code

204

In [22]:
response = requests.get('http://0.0.0.0:8080/ml_api/all_models')
response.json()['boston_rf_2']

{'deleted': True,
 'model': 'RandomForestRegressor',
 'params': {'bootstrap': True,
  'ccp_alpha': 0.0,
  'criterion': 'mse',
  'max_depth': 10,
  'max_features': 'auto',
  'max_leaf_nodes': None,
  'max_samples': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'min_samples_leaf': 3,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'n_estimators': 200,
  'n_jobs': None,
  'oob_score': False,
  'random_state': None,
  'verbose': 0,
  'warm_start': False},
 'retrained': False}

При попытке вызвать какой-либо метод для удаленной модели выпадает ошибка 404. Например:

In [23]:
response = requests.get('http://0.0.0.0:8080/ml_api/boston_rf/2')
response.status_code, response.text

(404, 'Model boston_rf of version 2 was deleted')

## Monitoring

Мониторинг приложения осуществляется посредством `PrometheusMetrics`, который логирует набор дефолтных метрик, а также ряд кастомных, в частности:
1. __by_status_counter__ – счетчик запросов по статусу выполнения (сколько было сделано запросов с тем или иным статусом выполнения)
2. __cnt_get_all_models__ – счетчик __get__ запроса к ручке __/ml_api/all_models__ (сколько раз смотрели словарь всех обученных моделей)
3. __cnt_get_models__ – счетчик __get__ запроса к ручке __/ml_api__ (сколько раз смотрели доступные классы моделей)
4. __cnt_get_preds__ – счетчик __get__ запроса к ручке __/ml_api/<model_name>/<model_version>__ по имени модели и статусу (сколько раз делали предсказания для той или иной модели любой версии с тем или иным статусом выполнения запроса – удалось получить предсказания или была какая-то ошибка)
5. __cnt_deletes__ – счетчик __delete__ запроса к ручке __/ml_api/<model_name>/<model_version>__ по имени модели и статусу (сколько раз удаляли версии той или иной модели с тем или иным статусом выполнения запроса – удалось удалить модель или была какая-то ошибка)
6. __cnt_trains__ – summary (кол-во запросов и время выполнения) __post__ запроса к ручке __/ml_api/<model_name>/<model_version>__ по имени модели и статусу (сколько обучено версий той или иной модели с тем или иным статусом выполнения запроса – удалось обучить модель или была какая-то ошибка)
7. __cnt_estimator_uses__ – summary (кол-во запросов и время выполнения) __post__ запроса к ручке __/ml_api/<model_name>/<model_version>__ по алгоритму и статусу (сколько раз для обучения модели использован той или иной алгоритм (estimator) с тем или иным статусом выполнения запроса – удалось обучить модель или была какая-то ошибка)
8. __cnt_retrains__ – summary (кол-во запросов и время выполнения) __put__ запроса к ручке __/ml_api/<model_name>/<model_version>__ по имени модели и статусу (сколько переобучений было для той или иной модели с тем или иным статусом выполнения запроса – удалось обучить модель на новых данных или была какая-то ошибка)

Метрики собираются каждые 5 секунд, отследить их можно по:
- http://127.0.0.1:8080/metrics
- http://127.0.0.1:9090 – Prometheus
- http://127.0.0.1:3000 – Grafana

На последнем ресурсе можно построить красивый дэшборд для визуализации метрик ))

In [28]:
requests.get('http://0.0.0.0:8080/metrics').text.split('\n')

['# HELP python_gc_objects_collected_total Objects collected during gc',
 '# TYPE python_gc_objects_collected_total counter',
 'python_gc_objects_collected_total{generation="0"} 83997.0',
 'python_gc_objects_collected_total{generation="1"} 16537.0',
 'python_gc_objects_collected_total{generation="2"} 70.0',
 '# HELP python_gc_objects_uncollectable_total Uncollectable object found during GC',
 '# TYPE python_gc_objects_uncollectable_total counter',
 'python_gc_objects_uncollectable_total{generation="0"} 0.0',
 'python_gc_objects_uncollectable_total{generation="1"} 0.0',
 'python_gc_objects_uncollectable_total{generation="2"} 0.0',
 '# HELP python_gc_collections_total Number of times this generation was collected',
 '# TYPE python_gc_collections_total counter',
 'python_gc_collections_total{generation="0"} 495.0',
 'python_gc_collections_total{generation="1"} 45.0',
 'python_gc_collections_total{generation="2"} 3.0',
 '# HELP python_info Python platform information',
 '# TYPE python_info